## TODO
* Features:
    * Easy visualization of: case starts, concurrent case obligations
    * printouts/reports of parameters used to generate analyses
    * utils: determine if an event falls in an arbitrary time window; aggregate to show a weekly incidence (across days of week), vs 52 weeks/yr, 12 months/year; expected formats for activity events 
    * some clinical questions to be answered: mishaps during nights and wkends vs weekdays, per provider, or likelihood when activity (anesthesia demand) is high

## Next
* helper function(s) to crank through cases, events, meds, and staffing, and create a collection of AnesthesiaCase objects with associated events and meds

In [1]:
import sys
sys.path.append("/home/tj/PycharmProjects/pytiva")

import os
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from tqdm import tqdm

import pytiva

In [ ]:
wd = r'/home/tj/PycharmProjects/pytiva/tests/test_data'
filename = 'OBCapacity_Output_20221116.xlsx'

#pytiva.anesthesia.AnesthesiaCaseEventsDataSet(pd.read_csv(os.path.join(wd, 'event_test_data.csv')))

df_meds = pd.read_excel(os.path.join(wd, filename), sheet_name='Events')
#df_events.rename(columns=col_map, inplace=True)
#event_ds = pytiva.anesthesia.AnesthesiaCaseEventsDataSet(df_events)
event_ds

In [ ]:
ds = event_ds.activity_ds_from_start_and_end_events(**NEURAXIAL_EVENT_PARAMS)

In [ ]:
_dose_units = [
        'g',
        'grams',
        'mg',
        'milligrams',
        'mcg',
        'micrograms',
        'ml',
        'milliliters',
        'l',
        'liters',
        'u',
        'units',
        'mu',
        'milliunits'
    ]
_dose_infusion_times = []

In [ ]:
ds.to_csv(os.path.join(wd, 'neuraxial_labor_activity_dataset_test.csv'), index=False)

In [ ]:
read_in = pd.read_csv(os.path.join(wd, 'neuraxial_labor_activity_dataset_test.csv'))
all(ads._df == pytiva.ActivityDataSet(read_in)._df)

In [ ]:
ads['duration'].describe().to_dict()

In [ ]:
ads

In [ ]:
## TODO: obfuscate and set aside med test data
## TODO: verboten columns